# w_05 | Plotly Dash y Express

* El objetivo de este notebook es construir iterativamente un dashboard de Coronavirus en Uruguay.
* Links a los datos
    * Estadísticas nacionales: https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY.csv
    * Estadísticas por departamento: https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY_porDepto_detalle.csv
    * Estadísticas de fallecimientos (edad y departamento): https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY_fallecimientos.csv
    * Estadísticas de CTI: https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY_cti.csv
    * Estadísticas de vacunación: https://catalogodatos.gub.uy/dataset/e766fbf7-0cc5-4b9a-a093-b56e91e88133/resource/5c549ba0-126b-45e0-b43f-b0eea72cf2cf/download/actos_vacunales.csv

* Vamos a usar Dash Bootstrap Components, un paquete que extiende Dash con nuevos componentes y stylesheets.
    * Más funcionalidad.
    * Más control.
* Además instalamos la última versión de Plotly.

In [2]:
%%capture
!pip install dash-bootstrap-components plotly dash jupyter-dash

In [3]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px
from dash.dependencies import Input, Output
import datetime as dt
import pandas as pd

<ipython-input-3-0d00bcd0541a>:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-3-0d00bcd0541a>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


* Leamos todos los datos.
* Recordar que al estar fuera de funciones, estos dataframes están en el "global state" y pueden ser usados en cualquier callback.

In [4]:
nacional = pd.read_csv("https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY.csv",
                       index_col=0, parse_dates=True, dayfirst=True)
#deptos = pd.read_csv("https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY_porDepto_detalle.csv",
#                           index_col=0, parse_dates=True, dayfirst=True)
#muertes = pd.read_csv("https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY_fallecimientos.csv",
#                             index_col=0, parse_dates=True, dayfirst=True)
#cti = pd.read_csv("https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY_cti.csv",
#                             index_col="fecha", parse_dates=True, dayfirst=True)
vacunas = pd.read_csv("https://catalogodatos.gub.uy/dataset/e766fbf7-0cc5-4b9a-a093-b56e91e88133/resource/5c549ba0-126b-45e0-b43f-b0eea72cf2cf/download/actos_vacunales.csv",
                        sep=";", index_col=0, parse_dates=True, dayfirst=True).sort_index()

In [5]:
nacional.head()

,dia,cantPersonasConInfeccionEnCurso,cantCasosNuevosAjustado,cantCasosNuevosConsolidado,cantCasosNuevosOriginal,acumCasos,cantFallecidos,acumFallecidos,cantCTI,cantCI,cantRecuperados,acumRecuperados,cantTest,acumTest,DIA,Egresades,reportadosFueraFecha,Positividad
fecha,,,,,,,,,,,,,,,,,,
2020-03-25,13,217,28,28,28,217,0,0,4,2,0,0,320,1858,3_MIERCOLES,NaN,NaN,0.088
2020-03-26,14,238,21,21,21,238,0,0,4,2,0,0,245,2103,4_JUEVES,NaN,NaN,0.086
2020-03-27,15,274,36,36,36,274,0,0,8,0,0,0,451,2554,5_VIERNES,NaN,NaN,0.080
2020-03-28,16,303,30,30,30,304,1,1,8,5,0,0,372,2926,6_SABADO,NaN,NaN,0.081
2020-03-29,17,309,6,6,6,309,0,1,10,2,0,0,200,3126,7_DOMINGO,NaN,NaN,0.030


## Ejemplo 1: creación de gráfico

In [ ]:
stylesheet = [dbc.themes.BOOTSTRAP]

#Agrega una etiqueta meta ("viewport") al HTML de la aplicación para controlar cómo se ajusta y escala el contenido en diferentes dispositivos.
#En este caso, asegura que el contenido se ajuste al ancho del dispositivo y tenga una escala inicial de 1, sin permitir ajustes por el usuario.
app = JupyterDash(__name__, external_stylesheets=stylesheet,
                  meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1, shrink-to-fit=no"}])


app.layout = html.Div([html.H1("Monitor COVID-19 en Uruguay"), #título
                       html.Div(f"Actualización: {dt.date.today().strftime('%d-%m-%y')}"),
                       html.Br(), #salto de linea
                       dcc.Graph(figure=px.line(data_frame=nacional, y=["cantCasosNuevosConsolidado",
                                                                        "cantCasosNuevosOriginal"]))],
                     className="m-3") #margen mediano

if __name__ == '__main__':
    app.run_server(debug=True,mode="external")

<ipython-input-4-f8effc934d8d>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-4-f8effc934d8d>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/usr/local/lib/python3.10/dist-packages/dash/dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


Dash app running on:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

* Usamos la stylesheet de Bootstrap.
    * [Hay otras disponibles](https://dash-bootstrap-components.opensource.faculty.ai/docs/themes/explorer/)
* Al usar esta librería podemos definir estilos en el parámetro `className` de los componentes.
    * Los nombres siguen la [nomenclatura de Bootstrap](https://getbootstrap.com/docs/5.1/getting-started/introduction/)
    * La idea es minimzar la necesidad de ajustar detalles de esta forma.
        * En el caso anterior, al `div` que engloba todo el layout le aplicamos la `class` "m-3", que aplica un margen de 3 en todas direcciones.

## Ejemplo 2: creación de tablas

* Incorporemos un recuadro con las estadísticas más recientes.
* El gráfico era temporal, por lo que reescribimos todo el layout.

In [5]:
latest_nat = nacional.iloc[-1][["cantCasosNuevosConsolidado", "cantRecuperados", "cantFallecidos", "cantCTI", "cantTest"]]
latest_vac = vacunas.iloc[-1][["Total Dosis 1", "Total Dosis 2"]].sum()
latest_nat["Positividad"] = round(latest_nat["cantCasosNuevosConsolidado"] / latest_nat["cantTest"] * 100, 1)
latest = latest_nat
latest["Vacunación"] = latest_vac
latest.index = ["Casos nuevos", "Recuperados", "Fallecidos", "CTI", "Tests", "Positividad", "Vacunados"]
latest = latest.to_frame().T
latest

,Casos nuevos,Recuperados,Fallecidos,CTI,Tests,Positividad,Vacunados
2022-04-17,183,324,4,18,2072,8.8,6


In [6]:
stylesheet = [dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__, external_stylesheets=stylesheet,
                  meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1, shrink-to-fit=no"}])

app.layout = dbc.Container([html.H1("Monitor COVID-19 en Uruguay"),
                            html.Div(f"Actualización: {latest.index[0].strftime('%d-%m-%y')}"),
                            html.Br(),
                            # Crea una fila que contendrá elementos de la interfaz.
                            dbc.Row(
                                # Crea una columna que contendrá un elemento de la interfaz. Aquí, se coloca una tabla generada a partir de un DataFrame (latest) utilizando dbc.Table.from_dataframe.
                                dbc.Col(
                                    #striped=True se establece en la tabla, se aplican rayas alternas de fondo
                                    # hover=True se establece en la tabla, las filas resaltarán cuando el cursor del mouse pase sobre ellas.
                                    # bordered=True, se agregan bordes a la tabla y a las celdas
                                    # responsive=True la tabla se adaptará automáticamente al tamaño de pantalla
                                    dbc.Table.from_dataframe(latest, striped=True, hover=True,
                                                             bordered=True, responsive=True),
                                #Define el ancho de la columna en diferentes tamaños de pantalla
                                width=10, md=8
                                ),
                                justify="center")],
                           fluid=True) # fluid=True indica que el contenedor ocupará todo el ancho disponible.


if __name__ == '__main__':
    app.run_server(debug=True, mode="external")

Dash app running on:


/usr/local/lib/python3.10/dist-packages/dash/dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

* En este layout empezamos a ver algunas cosas típicas de Bootstrap.
* Bootstrap está organizado en un grid de 12 espacios horizontales.
    * En cada línea caben 3 elementos de ancho 4, o 6 de ancho 2, etc, etc.
* Todo va envuelto en un `dbc.Container()` con `fluid=True`.
* El grid se distribuye en filas (rows) y columnas (columns).
    * En general el contenido va dentro de columns, que a su vez van dentro de rows.
        * `justify="center"` en `dbc.Row()` centra las columnas.
        * `width=10` indica que la `dbc.Col()` debe ocupar 10 espacios (en todos los tamaños de pantalla) de su parent element, que es una `dbc.Row()`.
            * `md=8` indica que en tamaños de pantalla medianos tiene que ocupar 8 espacios.
* Este setup permite controlar fácilmente la disposición de los elementos y que el layout sea responsive ante cambios en el viewport.

* Notar que ahora sacamos la fechas del dataframe

## Ejemplo 3: creación de tablas y figuras

*  Pongamos contenido a las tabs.

In [6]:
latest_nat = nacional.iloc[-1][["cantCasosNuevosConsolidado", "cantRecuperados", "cantFallecidos", "cantCTI", "cantTest"]]
latest_vac = vacunas.iloc[-1][["Total Dosis 1", "Total Dosis 2"]].sum()
latest_nat["Positividad"] = round(latest_nat["cantCasosNuevosConsolidado"] / latest_nat["cantTest"] * 100, 1)
latest = latest_nat
latest["Vacunación"] = latest_vac
latest.index = ["Casos nuevos", "Recuperados", "Fallecidos", "CTI", "Tests", "Positividad", "Vacunados"]
latest = latest.to_frame().T
latest

,Casos nuevos,Recuperados,Fallecidos,CTI,Tests,Positividad,Vacunados
2022-04-17,183,324,4,18,2072,8.8,6


In [7]:
stylesheet = [dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__, external_stylesheets=stylesheet,
                  meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1, shrink-to-fit=no"}])

# generar gráficos de área para visualizar la distribución o evolución de los casos nuevos consolidados y los casos acumulados a lo largo del tiempo
fig_casos_nuevos = px.area(data_frame=nacional, y="cantCasosNuevosConsolidado")
fig_casos_acum = px.area(data_frame=nacional, y="acumCasos")

# crea un diseño de dos columnas que muestra dos gráficos lado a lado dentro de una fila en una pestaña específica.
#Los gráficos incluidos son fig_casos_nuevos y fig_casos_acum, que visualizan casos nuevos consolidados y casos acumulados respectivamente.
tab_1 = [dbc.Row([
dbc.Col(dcc.Graph(figure=fig_casos_nuevos), md=6),
dbc.Col(dcc.Graph(figure=fig_casos_acum), md=6),
])]

# gráficos de área para visualizar la distribución de fallecimientos y la evolución acumulada de los fallecimientos a lo largo del tiempo
fig_fallecidos = px.area(data_frame=nacional, y="cantFallecidos")
fig_fallecidos_acum = px.area(data_frame=nacional, y="acumFallecidos")

# crea un diseño de dos columnas que muestra dos gráficos lado a lado dentro de una fila en una pestaña específica.
#Los gráficos incluidos son fig_fallecidos y fig_fallecidos_acum,
tab_2 = [dbc.Row([
dbc.Col(dcc.Graph(figure=fig_fallecidos), md=6),
dbc.Col(dcc.Graph(figure=fig_fallecidos_acum), md=6),
])]

# gráficos de área para visualizar la distribución o evolución de la cantidad de pruebas realizadas y la positividad de las pruebas a lo largo del tiempo
fig_tests = px.area(data_frame=nacional, y="cantTest")
fig_positividad = px.area(data_frame=nacional, y="Positividad")

# crea un diseño de dos columnas que muestra dos gráficos lado a lado dentro de una fila en una pestaña específica.
#Los gráficos incluidos son fig_tests y fig_positividad, que visualizan la cantidad de pruebas realizadas y la positividad de las pruebas, respectivamente.
tab_3 = [dbc.Row([
dbc.Col(dcc.Graph(figure=fig_tests), md=6),
dbc.Col(dcc.Graph(figure=fig_positividad), md=6),
])]

app.layout = dbc.Container([
    html.H1("Monitor COVID-19 en Uruguay"),  # Título principal
    html.Div(f"Actualización: {latest.index[0].strftime('%d-%m-%y')}"),  # Fecha de actualización
    html.Br(),  # Salto de línea

    # Tarjeta que muestra los últimos datos en forma de tabla
    dbc.Card([
        dbc.CardHeader(html.H5("Últimos datos")),  # Encabezado de la tarjeta
        dbc.CardBody(
            dbc.Table.from_dataframe(latest, striped=True, hover=True, bordered=True, responsive=True)
        )  # Cuerpo de la tarjeta que contiene la tabla
    ], color="primary", outline=True),  # Estilo y borde de la tarjeta

    html.Br(),  # Salto de línea

    # Pestañas que contienen los gráficos definidos previamente en 'tab_1', 'tab_2' y 'tab_3'
    dbc.Tabs([
        dbc.Tab(tab_1, label="Casos"),  # Pestaña para visualizar casos
        dbc.Tab(tab_2, label="Fallecimientos"),  # Pestaña para visualizar fallecimientos
        dbc.Tab(tab_3, label="Tests")  # Pestaña para visualizar pruebas
    ])
], fluid=True)


if __name__ == '__main__':
    app.run_server(debug=True, mode="external")

/usr/local/lib/python3.10/dist-packages/dash/dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


<IPython.core.display.Javascript object>

Dash app running on:


<IPython.core.display.Javascript object>

## Ejemplo 4: Caso avanzado, con un solo flujo de trabajo

In [ ]:
%%capture
!pip install dash-bootstrap-components plotly==5.2.1 dash jupyter-dash

In [ ]:
# Importación de las librerías necesarias
import plotly.io as pio
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output

# Configuración de la hoja de estilos de Bootstrap
stylesheet = [dbc.themes.BOOTSTRAP]

# Lectura de datos relacionados con el COVID-19 en Uruguay desde URLs
nacional = pd.read_csv("https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY.csv",
                       index_col=0, parse_dates=True, dayfirst=True)
vacunas = pd.read_csv("https://catalogodatos.gub.uy/dataset/e766fbf7-0cc5-4b9a-a093-b56e91e88133/resource/5c549ba0-126b-45e0-b43f-b0eea72cf2cf/download/actos_vacunales.csv",
                        sep=";", index_col=0, parse_dates=True, dayfirst=True).sort_index()

# Selección de los últimos datos de casos y vacunación
latest_nat = nacional.iloc[-1][["cantCasosNuevosConsolidado", "cantRecuperados", "cantFallecidos", "cantCTI", "cantTest"]] # Seleccionar los últimos datos de casos
latest_vac = vacunas.iloc[-1][["Total Dosis 1", "Total Dosis 2"]].sum() # Seleccionar los últimos datos de vacunación
latest_nat["Positividad"] = round(latest_nat["cantCasosNuevosConsolidado"] / latest_nat["cantTest"] * 100, 1) # Calcular positividad de casos
latest = latest_nat # Datos combinados de casos y vacunación
latest["Vacunación"] = latest_vac
latest.index = ["Casos nuevos", "Recuperados", "Fallecidos", "CTI", "Tests", "Positividad", "Vacunados"] # Etiquetas para los datos
latest = latest.to_frame().T

# Configuración del template de la visualización
pio.templates.default = "plotly_white"

# Creación de la aplicación Dash
app = JupyterDash(__name__, external_stylesheets=stylesheet,
                  meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1, shrink-to-fit=no"}])

# Función para actualizar el diseño de las figuras
def update_fig_layout(fig):
    fig.update_layout(legend=dict(yanchor="top", y=-0.1,
                                  xanchor="left", x=0.0,
                                  title_text="",
                                  orientation="h"),
                      xaxis={"title_text": ""},
                      yaxis={"title_text": ""},
                      font_family="Helvetica",
                      font_color="#8C8C8C")
    return

# Creación de gráficos de casos acumulados y fallecimientos acumulados
fig_casos_acum = px.area(data_frame=nacional, y="acumCasos",
                         title="Casos acumulados", color_discrete_sequence=px.colors.qualitative.Vivid)
update_fig_layout(fig_casos_acum)
fig_fallecidos_acum = px.area(data_frame=nacional, y="acumFallecidos", title="Fallecimientos acumulados",
                              color_discrete_sequence=px.colors.qualitative.Vivid)
update_fig_layout(fig_fallecidos_acum)

# Definición de pestañas y contenidos
tab_1 = [html.Br(),
         dbc.Row([
             dbc.Col(
                 dbc.Select(id="rolling-select-casos", options=[{"label": "No agregar promedio", "value": "0"},
                                                          {"label": "7 días", "value": "P7"},
                                                          {"label": "14 días", "value": "P14"}],
                            placeholder="Agregar promedio"), width=6)]),
         dbc.Row([
             dbc.Col(dcc.Graph(id="chart-casos-nuevos"), md=6),
             dbc.Col(dcc.Graph(figure=fig_casos_acum), md=6),
], className="g-0")]


tab_2 = [html.Br(),
         dbc.Row([
             dbc.Col(
                 dbc.Select(id="rolling-select-fallecidos", options=[{"label": "No agregar promedio", "value": "0"},
                                                          {"label": "7 días", "value": "P7"},
                                                          {"label": "14 días", "value": "P14"}],
                            placeholder="Agregar promedio"), width=6)]),
         dbc.Row([
             dbc.Col(dcc.Graph(id="chart-fallecidos"), md=6),
             dbc.Col(dcc.Graph(figure=fig_fallecidos_acum), md=6),
], className="g-0")]

tab_3 = [html.Br(),
         dbc.Row([
             dbc.Col(
                 dbc.Select(id="rolling-select-tests", options=[{"label": "No agregar promedio", "value": "0"},
                                                          {"label": "7 días", "value": "P7"},
                                                          {"label": "14 días", "value": "P14"}],
                            placeholder="Agregar promedio"), width=6)]),
         dbc.Row([
             dbc.Col(dcc.Graph(id="chart-tests"), md=6),
             dbc.Col(dcc.Graph(id="chart-positividad"), md=6),
], className="g-0")]

# Diseño general de la aplicación
app.layout = dbc.Container([html.H1("Monitor COVID-19 en Uruguay"),
                            html.Div(f"Actualización: {latest.index[0].strftime('%d-%m-%y')}"),
                            html.Br(),
                            dbc.Card([dbc.CardHeader(html.H5("Últimos datos")),
                                      dbc.CardBody(
                                          dbc.Table.from_dataframe(latest, striped=True, hover=True,
                                                             bordered=True, responsive=True))],
                                     color="primary", outline=True),
                            html.Br(),
                            dbc.Tabs([dbc.Tab(tab_1, label="Casos"),
                                      dbc.Tab(tab_2, label="Fallecimientos"),
                                      dbc.Tab(tab_3, label="Tests")])],
                           fluid=True)

# Callbacks para actualizar los gráficos en respuesta a la interacción del usuario
@app.callback(Output("chart-casos-nuevos", "figure"), [Input("rolling-select-casos", "value")])
def crear_grafico_casos(promedio, data=nacional):
    if not promedio or promedio == "0":
        fig = px.line(data_frame=data, y="cantCasosNuevosConsolidado", title="Casos nuevos",
                      color_discrete_sequence=px.colors.qualitative.Vivid)
    else:
        data["P7"] = data["cantCasosNuevosConsolidado"].rolling(7).mean()
        data["P14"] = data["cantCasosNuevosConsolidado"].rolling(14).mean()
        fig = px.line(data_frame=data, y=["cantCasosNuevosConsolidado"] + [promedio], title="Casos nuevos",
                      color_discrete_sequence=px.colors.qualitative.Vivid)
    update_fig_layout(fig)
    return fig


@app.callback(Output("chart-fallecidos", "figure"), [Input("rolling-select-fallecidos", "value")])
def crear_grafico_fallecidos(promedio, data=nacional):
    if not promedio or promedio == "0":
        fig = px.line(data_frame=data, y="cantFallecidos", title="Fallecimientos",
                      color_discrete_sequence=px.colors.qualitative.Vivid)
    else:
        data["P7"] = data["cantFallecidos"].rolling(7).mean()
        data["P14"] = data["cantFallecidos"].rolling(14).mean()
        fig = px.line(data_frame=data, y=["cantFallecidos"] + [promedio], title="Fallecimientos acumulados",
                      color_discrete_sequence=px.colors.qualitative.Vivid)
    update_fig_layout(fig)
    return fig


@app.callback(Output("chart-tests", "figure"), [Input("rolling-select-tests", "value")])
def crear_grafico_tests(promedio, data=nacional):
    if not promedio or promedio == "0":
        fig = px.line(data_frame=data, y="cantTest", color_discrete_sequence=px.colors.qualitative.Vivid,
                      title="Tests realizados")
    else:
        data["P7"] = data["cantTest"].rolling(7).mean()
        data["P14"] = data["cantTest"].rolling(14).mean()
        fig = px.line(data_frame=data, y=["cantTest"] + [promedio], color_discrete_sequence=px.colors.qualitative.Vivid,
                     title="Tests realizados")
    update_fig_layout(fig)
    return fig


@app.callback(Output("chart-positividad", "figure"), [Input("rolling-select-tests", "value")])
def crear_grafico_positividad(promedio, data=nacional):
    aux = data.copy()
    if not promedio or promedio == "0":
        fig = px.line(data_frame=aux, y="Positividad", color_discrete_sequence=px.colors.qualitative.Vivid,
                      title="Positividad")
    else:
        aux["P7"] = aux["Positividad"].rolling(7).mean()
        aux["P14"] = aux["Positividad"].rolling(14).mean()
        fig = px.line(data_frame=aux, y=["Positividad"] + [promedio], color_discrete_sequence=px.colors.qualitative.Vivid,
                     title="Positividad")
    update_fig_layout(fig)
    return fig

# Iniciar el servidor de la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, mode="external")

<ipython-input-2-7f09a216e7d7>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-7f09a216e7d7>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Dash app running on:


/usr/local/lib/python3.10/dist-packages/dash/dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Resumen del código


1. **Importación de Librerías:**
   - El código comienza importando las librerías necesarias para el funcionamiento de la aplicación. Cada librería tiene un propósito específico:
     - `plotly.io as pio`: Importa la funcionalidad para configurar aspectos de las visualizaciones de Plotly.
     - `pandas as pd`: Permite la manipulación y análisis de datos en forma de estructuras de datos tabulares (DataFrames).
     - `dash_core_components as dcc` y `dash_html_components as html`: Proporcionan los componentes necesarios para construir la interfaz gráfica.
     - `dash_bootstrap_components as dbc`: Permite utilizar componentes Bootstrap para una mejor presentación visual.
     - `plotly.express as px`: Facilita la creación de gráficos interactivos con Plotly.
     - `JupyterDash`: Extiende la funcionalidad de Dash para funcionar en el entorno de Jupyter.
     - `dash.dependencies`: Proporciona funcionalidad para manejar las dependencias de entrada y salida de los callbacks.

2. **Configuración de Estilos y Lectura de Datos:**
   - Se define la variable `stylesheet` con el tema Bootstrap que se utilizará para los estilos visuales de la aplicación.
   - Los datos relacionados con el COVID-19 en Uruguay se leen desde URLs utilizando la función `pd.read_csv()`. Los datos son obtenidos en forma de DataFrames, lo que permite su manipulación y análisis.

3. **Selección de Últimos Datos:**
   - Se seleccionan los últimos datos de casos, fallecimientos y vacunación a partir de los DataFrames de datos nacionales.
   - Se calcula la positividad de los casos nuevos como el porcentaje de casos respecto al número de pruebas realizadas. Esto se agrega al conjunto de datos para su posterior visualización.

4. **Configuración del Template de Visualización:**
   - Se establece el template de visualización de Plotly para que tenga un fondo blanco.

5. **Creación de la Aplicación Dash:**
   - Se crea una instancia de la aplicación Dash utilizando `JupyterDash()`. Se proporcionan los estilos externos y se configuran metaetiquetas para la vista y el escalado de la interfaz.

6. **Función para Actualizar el Diseño de Figuras:**
   - Se define una función llamada `update_fig_layout()` que acepta una figura de Plotly como entrada. Esta función se encarga de personalizar el diseño de la figura, ajustando aspectos como la posición de la leyenda, los títulos de los ejes y el estilo de la fuente.

7. **Creación de Gráficos Iniciales:**
   - Se crean gráficos de áreas utilizando Plotly Express para mostrar los casos acumulados y los fallecimientos acumulados. Luego de crear cada figura, se llama a la función `update_fig_layout()` para aplicar el diseño personalizado.

8. **Definición de Contenidos de Pestañas:**
   - Se definen tres conjuntos de contenido para las pestañas "Casos", "Fallecimientos" y "Tests". Cada conjunto incluye componentes HTML y gráficos interactivos que permiten a los usuarios seleccionar opciones y explorar los datos.

9. **Diseño General de la Aplicación:**
   - Se crea el diseño general de la aplicación utilizando `dbc.Container()`. Esto incluye un encabezado con el título "Monitor COVID-19 en Uruguay", la fecha de la última actualización de datos, una tarjeta con la tabla de los últimos datos y pestañas que contienen los contenidos definidos previamente.

10. **Callbacks para Actualización de Gráficos:**
    - Se definen una serie de funciones de callback que se activarán cuando cambien ciertos valores en la interfaz. Por ejemplo, cuando el usuario seleccione una opción de promedio móvil para los datos de casos, se activará el callback correspondiente. Estas funciones actualizan los gráficos con los nuevos datos y ajustan su diseño.

11. **Iniciar el Servidor de la Aplicación:**
    - Finalmente, se verifica si el script se está ejecutando como el programa principal. Si es así, se inicia el servidor de la aplicación Dash utilizando `app.run_server()` con la opción de depuración activada y en modo "external".



###Explicando elementos:


In [ ]:
def update_fig_layout(fig):
    fig.update_layout(legend=dict(yanchor="top", y=-0.1,
                                  xanchor="left", x=0.0,
                                  title_text="",
                                  orientation="h"),
                      xaxis={"title_text": ""},
                      yaxis={"title_text": ""},
                      font_family="Helvetica",
                      font_color="#8C8C8C")
    return

`def update_fig_layout(fig)`: Esto define una función llamada `update_fig_layout` que toma como argumento una figura de Plotly (fig).

`fig.update_layout(...)`: Aquí se llama al método update_layout() en la figura fig para aplicar cambios al diseño de la figura.

`legend=dict(yanchor="top", y=-0.1, xanchor="left", x=0.0, title_text="", orientation="h")`: Esto configura la leyenda de la figura. Los parámetros especificados ajustan su posición (yanchor, xanchor, y, x), eliminan el título (title_text) y orientan la leyenda horizontalmente (orientation).

`xaxis={"title_text": ""}`: Define la configuración del eje x de la figura. En este caso, se elimina el título (title_text) del eje x.

`yaxis={"title_text": ""}`: Similar al eje x, esto elimina el título del eje y.

`font_family="Helvetica"`: Establece la fuente del texto en la figura como "Helvetica".

`font_color="#8C8C8C"`: Define el color de la fuente en la figura como un gris claro.

`return`: La función no devuelve ningún valor explícitamente. En Python, si una función no tiene una declaración de retorno explícita, devuelve None por defecto.

In [ ]:
# Creación de gráficos de casos acumulados y fallecimientos acumulados
fig_casos_acum = px.area(data_frame=nacional, y="acumCasos",
                         title="Casos acumulados", color_discrete_sequence=px.colors.qualitative.Vivid)
update_fig_layout(fig_casos_acum)


`fig_casos_acum`: Se crea una variable llamada fig_casos_acum para almacenar el gráfico de casos acumulados.

`px.area(...)`: Aquí se utiliza la función px.area() de Plotly Express para crear un gráfico de área. Se le proporciona como argumento el DataFrame nacional y se configura el eje y como "acumCasos", que representa los casos acumulados. El parámetro title establece el título del gráfico como "Casos acumulados". El argumento color_discrete_sequence configura la paleta de colores a utilizar para las áreas.

`update_fig_layout(fig_casos_acum)`: Esta línea llama a la función `update_fig_layout()` que habíamos discutido anteriormente. Esto aplica personalización adicional al diseño de la figura fig_casos_acum.

In [ ]:
fig_fallecidos_acum = px.area(data_frame=nacional, y="acumFallecidos", title="Fallecimientos acumulados",
                              color_discrete_sequence=px.colors.qualitative.Vivid)
update_fig_layout(fig_fallecidos_acum)

Similar al caso anterior, se crea una variable fig_fallecidos_acum para almacenar el gráfico de fallecimientos acumulados.

Se utiliza nuevamente `px.area(`) para crear un gráfico de área, esta vez configurando el eje y como "acumFallecidos" para representar los fallecimientos acumulados. El título se establece como "Fallecimientos acumulados" y se configura la paleta de colores.

Al igual que antes, se llama a `update_fig_layout()` para personalizar el diseño del gráfico `fig_fallecidos_acum`.

In [ ]:
# Definición de pestañas y contenidos
tab_1 = [html.Br(),
         dbc.Row([
             dbc.Col(
                 dbc.Select(id="rolling-select-casos", options=[{"label": "No agregar promedio", "value": "0"},
                                                          {"label": "7 días", "value": "P7"},
                                                          {"label": "14 días", "value": "P14"}],
                            placeholder="Agregar promedio"), width=6)]),
         dbc.Row([
             dbc.Col(dcc.Graph(id="chart-casos-nuevos"), md=6),
             dbc.Col(dcc.Graph(figure=fig_casos_acum), md=6),
], className="g-0")]


tab_2 = [html.Br(),
         dbc.Row([
             dbc.Col(
                 dbc.Select(id="rolling-select-fallecidos", options=[{"label": "No agregar promedio", "value": "0"},
                                                          {"label": "7 días", "value": "P7"},
                                                          {"label": "14 días", "value": "P14"}],
                            placeholder="Agregar promedio"), width=6)]),
         dbc.Row([
             dbc.Col(dcc.Graph(id="chart-fallecidos"), md=6),
             dbc.Col(dcc.Graph(figure=fig_fallecidos_acum), md=6),
], className="g-0")]

tab_3 = [html.Br(),
         dbc.Row([
             dbc.Col(
                 dbc.Select(id="rolling-select-tests", options=[{"label": "No agregar promedio", "value": "0"},
                                                          {"label": "7 días", "value": "P7"},
                                                          {"label": "14 días", "value": "P14"}],
                            placeholder="Agregar promedio"), width=6)]),
         dbc.Row([
             dbc.Col(dcc.Graph(id="chart-tests"), md=6),
             dbc.Col(dcc.Graph(id="chart-positividad"), md=6),
], className="g-0")]


Se definen tres pestañas: **"Casos", "Fallecimientos"** y **"Tests"**.
Cada pestaña (`tab_1`, `tab_2` y `tab_3`) es una lista que contiene elementos `HTML` y componentes Dash que conformarán el contenido de la pestaña correspondiente.

Cada pestaña tiene una estructura similar:
`html.Br()`: Esto agrega un espacio en blanco (línea en blanco) entre elementos.
`dbc.Row([...])`: Define una fila de elementos utilizando componentes de `Bootstrap (dbc.Col)`. En este caso, se utiliza para organizar el contenido en una estructura de fila.

`dbc.Col(...):` Define una columna para organizar elementos dentro de la fila.


 Se especifica el contenido de cada columna utilizando componentes `Dash (dcc.Graph, dbc.Select, etc.)`.

`dcc.Graph(id="...", ...)` y `dbc.Select(id="...", ...)`: Aquí se crean componentes Dash de tipo gráfico y selección, respectivamente. Cada uno tiene un ID único que se utiliza para vincular con los callbacks más adelante.

`dbc.Col(dcc.Graph(...), md=6)`: Establece una columna con una anchura media `(md=6)` que contiene un gráfico.

`className="g-0"`: Define la clase de estilo "g-0", que significa que no habrá espacio (gutter) entre las columnas dentro de la fila. Esto crea un diseño ajustado.


Establecen la estructura de las pestañas "Casos", "Fallecimientos" y "Tests" en la aplicación Dash. Cada pestaña contiene componentes interactivos y gráficos para mostrar información relacionada con los casos, fallecimientos y pruebas de COVID-19 en Uruguay.

In [ ]:
# Diseño general de la aplicación
app.layout = dbc.Container([html.H1("Monitor COVID-19 en Uruguay"),
                            html.Div(f"Actualización: {latest.index[0].strftime('%d-%m-%y')}"),
                            html.Br(),
                            dbc.Card([dbc.CardHeader(html.H5("Últimos datos")),
                                      dbc.CardBody(
                                          dbc.Table.from_dataframe(latest, striped=True, hover=True,
                                                             bordered=True, responsive=True))],
                                     color="primary", outline=True),
                            html.Br(),
                            dbc.Tabs([dbc.Tab(tab_1, label="Casos"),
                                      dbc.Tab(tab_2, label="Fallecimientos"),
                                      dbc.Tab(tab_3, label="Tests")])],
                           fluid=True)

`app.layout = dbc.Container([...], fluid=True)`: Aquí se establece el diseño principal de la aplicación Dash dentro de un contenedor de Bootstrap. El argumento fluid=True indica que el contenedor debe ocupar todo el ancho disponible.

`html.H1("Monitor COVID-19 en Uruguay")`: Se crea un título principal para la aplicación, mostrando "Monitor COVID-19 en Uruguay".

`html.Div(f"Actualización: {latest.index[0].strftime('%d-%m-%y')}")`: Crea una división HTML que muestra la fecha de actualización de los datos. La fecha se extrae del índice de los últimos datos.

`html.Br()`: Agrega un espacio en blanco.

`dbc.Card([...], color="primary", outline=True)`: Crea un componente de tarjeta de Bootstrap. El argumento color="primary" establece el color de fondo de la tarjeta y outline=True agrega un borde. El contenido de la tarjeta está compuesto por un encabezado y un cuerpo.

`dbc.CardHeader(html.H5("Últimos datos"))`: Define el encabezado de la tarjeta con un título de nivel H5.

`dbc.CardBody([...])`: Define el cuerpo de la tarjeta. Contiene una tabla generada a partir de los últimos datos (latest) utilizando dbc.Table.from_dataframe(...). Los parámetros striped, hover, bordered y responsive personalizan el estilo y la interactividad de la tabla.

`html.Br()`: Agrega otro espacio en blanco.

`dbc.Tabs([...])`: Crea una serie de pestañas utilizando el componente dbc.Tab. Cada pestaña se asocia con uno de los conjuntos de contenido previamente definidos (`tab_1`, `tab_2` y `tab_3`). Se etiquetan como "Casos", "Fallecimientos" y "Tests" respectivamente.



In [ ]:
# Callbacks para actualizar los gráficos en respuesta a la interacción del usuario
@app.callback(Output("chart-casos-nuevos", "figure"), [Input("rolling-select-casos", "value")])
def crear_grafico_casos(promedio, data=nacional):
    if not promedio or promedio == "0":
        fig = px.line(data_frame=data, y="cantCasosNuevosConsolidado", title="Casos nuevos",
                      color_discrete_sequence=px.colors.qualitative.Vivid)
    else:
        data["P7"] = data["cantCasosNuevosConsolidado"].rolling(7).mean()
        data["P14"] = data["cantCasosNuevosConsolidado"].rolling(14).mean()
        fig = px.line(data_frame=data, y=["cantCasosNuevosConsolidado"] + [promedio], title="Casos nuevos",
                      color_discrete_sequence=px.colors.qualitative.Vivid)
    update_fig_layout(fig)
    return fig

`@app.callback(...):` Esta línea define un decorador que indica que la función que le sigue es un callback. El primer argumento de @app.callback es el componente de salida que se actualizará (en este caso, el gráfico de casos nuevos), y el segundo argumento es una lista de componentes de entrada que desencadenarán el callback (en este caso, la selección de tipo de promedio).

`Output("chart-casos-nuevos", "figure")`: Esto especifica que el gráfico con el ID "chart-casos-nuevos" tendrá su figura actualizada por el callback.

`Input("rolling-select-casos", "value")`: Esto especifica que el valor seleccionado en el componente de selección con el ID `"rolling-select-casos" `será el que active el callback.

`crear_grafico_casos(promedio, data=nacional):` La función crear_grafico_casos es el cuerpo del callback. Toma como argumentos el valor seleccionado (promedio) y el DataFrame nacional como datos por defecto.

`if not promedio or promedio == "0":`: Este bloque verifica si el valor seleccionado es nulo o "0" (que indica "No agregar promedio"). Si es así, crea un gráfico de líneas con los casos nuevos consolidados sin promedios.

`else:`: Si se selecciona algún promedio, calcula los promedios móviles de 7 y 14 días de los casos nuevos consolidados y crea un gráfico de líneas que muestra los casos nuevos consolidados y el promedio seleccionado.

`update_fig_layout(fig)`: Llama a la función `update_fig_layout()` para personalizar el diseño del gráfico antes de devolverlo.

`return fig`: Finalmente, la figura actualizada se devuelve como resultado del callback.

Los otros callbacks (crear_grafico_fallecidos, crear_grafico_tests, crear_grafico_positividad) funcionan de manera similar, actualizando los gráficos de fallecimientos, pruebas y positividad en respuesta a las selecciones de promedio realizadas por el usuario. Cada callback actualiza la figura correspondiente y aplica el diseño personalizado a través de la función update_fig_layout().

# Fin